# <center> Innoscape - Telecom Paristech - Projet fil rouge </center>
# <center> Mapping Groupes de produits </center>

In [16]:
# Librairies 
import pandas as pd
import numpy as np
from scipy import stats as scs
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import FrenchStemmer
import re
import random
from Preprocessing2 import *
import gensim.models as models
from gensim import corpora

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 60)

#chemin = "../Data/"
chemin = "C:/Users/JN/Desktop/Arbeit/fil_rouge/github/Data/"
chemin_val = "/Users/valentinphetchanpheng/Documents/MS BGD Telecom ParisTech/Projet Fil Rouge/"

C:\Users\JN\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Importation des données et reformatage

In [2]:
# Importation des données vendeurs
df_seller_origin = pd.read_csv(chemin + "t3_all_fr_prices_posweekly_prod_without_price.csv", sep = ",")

#Création d'un df allégé pour le travail - on ne retient que les colonnes identifiées comme utiles pour l'analyse à ce stade
columns_to_keep = ['t3_all_fr_prices_posweekly_prod.productfamily_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory4_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory3_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory2_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory1_seller',
                   't3_all_fr_prices_posweekly_prod.productseller_name',
                   't3_all_fr_prices_posweekly_prod.id_source',
                   't3_all_fr_prices_posweekly_prod.brand_seller']

df_seller = df_seller_origin[columns_to_keep]

#on raccourcit un peu les noms des colonnes en ôtant les préfixes redondants
shortened_names = map(lambda string: string.replace('t3_all_fr_prices_posweekly_prod.',''), columns_to_keep)
col_name_dict = dict(zip(columns_to_keep, shortened_names))
df_seller.rename(columns=col_name_dict, inplace=True)

In [3]:
# Importation des données références catégories Innoscape
df_ref_origin = pd.read_csv(chemin + "2018.11.21_innoscape_internal_architecture.csv")
df_ref = df_ref_origin.drop("Date", axis=1)

#On garde que la colonne correspondante à la famille de produit de référence finale
columns_to_keep = ['innoscape_ref_productfamily_seller']
df_ref = df_ref[columns_to_keep]
df_ref = df_ref.drop_duplicates()

In [4]:
df_seller = df_seller.dropna(how='all', subset=['productfamily_seller',
                                                'prodcategory4_seller',
                                                'prodcategory3_seller',
                                                'prodcategory2_seller',
                                                'prodcategory1_seller',
                                                'productseller_name'])
df_seller = df_seller.fillna("")

# Retrait des caractères spéciaux et de la ponctuation
dictionary = {'à':'a','â':'a','ä':'a',
              'é':'e','è':'e','ê':'e','ë':'e',
              'ï':'i','î':'i',
              'ô':'o','ö':'o',
              'ù':'u','û':'u',
              'ç':'c',
              ',':' ', "'":" ", ";":" ", ":":" ", "\?":" ", "!":" ", "\.":'', "  ":" ", 
              "\(":"","\)":"","\[":"","\]":"","-":" ","\+":"","/":"","\*":"","–":" ",
              "&":"et"
              }


In [5]:
stop_words = {'a',
             'ai',
             'aie',
             'aient',
             'aies',
             'ait',
             'as',
             'au',
             'aura',
             'aurai',
             'auraient',
             'aurais',
             'aurait',
             'auras',
             'aurez',
             'auriez',
             'aurions',
             'aurons',
             'auront',
             'aux',
             'avaient',
             'avais',
             'avait',
             'avec',
             'avez',
             'aviez',
             'avions',
             'avons',
             'ayant',
             'ayante',
             'ayantes',
             'ayants',
             'ayez',
             'ayons',
             'c',
             'ce',
             'ces',
             'd',
             'dans',
             'de',
             'des',
             'du',
             'elle',
             'en',
             'es',
             'est',
             'et',
             'eu',
             'eue',
             'eues',
             'eurent',
             'eus',
             'eusse',
             'eussent',
             'eusses',
             'eussiez',
             'eussions',
             'eut',
             'eux',
             'eûmes',
             'eût',
             'eûtes',
             'furent',
             'fus',
             'fusse',
             'fussent',
             'fusses',
             'fussiez',
             'fussions',
             'fut',
             'fûmes',
             'fût',
             'fûtes',
             'il',
             'j',
             'je',
             'l',
             'la',
             'le',
             'leur',
             'lui',
             'm',
             'ma',
             'mais',
             'me',
             'mes',
             'moi',
             'mon',
             'même',
             'n',
             'ne',
             'nos',
             'notre',
             'nous',
             'on',
             'ont',
             'ou',
             'par',
             'pas',
             'pour',
             'qu',
             'que',
             'qui',
             's',
             'sa',
             'se',
             'sera',
             'serai',
             'seraient',
             'serais',
             'serait',
             'seras',
             'serez',
             'seriez',
             'serions',
             'serons',
             'seront',
             'ses',
             'soient',
             'sois',
             'soit',
             'sommes',
             'son',
             'sont',
             'soyez',
             'soyons',
             'suis',
             'sur',
             't',
             'ta',
             'te',
             'tes',
             'toi',
             'ton',
             'tu',
             'un',
             'une',
             'vos',
             'votre',
             'vous',
             'y',
             'à',
             'étaient',
             'étais',
             'était',
             'étant',
             'étante',
             'étantes',
             'étants',
             'étiez',
             'étions',
             'été',
             'étée',
             'étées',
             'étés',
             'êtes',
             ',',
             '(',
             ')',
             '[',
             ']',
             '-',
             '/',
             '+',
             '*',
             '{',
             '}',
             '!',
             ';',
             ':',
             '?',
             '.',
             '&'
             }


In [6]:
df_ref=preprocessing(df_ref,df_ref.columns,dictionary, stop_words)

list_new_col = ['productfamily_seller', 'prodcategory4_seller', 'prodcategory3_seller', 'prodcategory2_seller', 'prodcategory1_seller', 'productseller_name']
df_seller = preprocessing(df_seller, list_new_col, dictionary, stop_words)

In [7]:
df_seller.head(10)

productfamily_seller   prodcategory4_seller  \
0     Colles à carrelage     Colles à carrelage   
1                                                 
2                                                 
3                 Mastic                 Mastic   
4                  Cires                  Cires   
5                                                 
6      Perceuse-visseuse              Perceuses   
7  Produits d'étanchéité  Produits d'étanchéité   
8              Arroseurs              Arroseurs   
9                                                 

               prodcategory3_seller                      prodcategory2_seller  \
0                Colles et adhésifs                             Quincaillerie   
1                                                                               
2  Accessoires pour lave-vaisselles                               Accessoires   
3                Colles et adhésifs                             Quincaillerie   
4  Peintures, teintures et solvants  Peintures, outils et traitement des murs   
5                                    Peintures, outils et traitement des murs   
6         Outillage électroportatif       Outillage à main et électroportatif   
7                Colles et adhésifs                             Quincaillerie   
8               Matériel d'arrosage                                 Jardinage   
9                                                                               

  prodcategory1_seller                                 productseller_name  \
0            Bricolage  SikaCeram Xtra - Colle à carrelage intérieur e...   
1                       Ariston 3200362 scaldabagno elettrico PRO 100 H/3   
2  Gros électroménager  AEG Panier À Couverts Lave-vaisselle Cage & Ha...   
3            Bricolage              Colle-mastic pour joints d'étanchéité   
4            Bricolage  Sikagard Protection Sol MAT - Imperméabilisant...   
5            Bricolage  Sikagard Protection Toiture inclinée - Protect...   
6            Bricolage  Perceuse-visseuse RYOBI 18V OnePlus - 1 batter...   
7            Bricolage  Sika Multiseal - Bande d'étanchéité autocollan...   
8               Jardin            Black and Decker 34345 Arroseur Impulse   
9                       Ryobi-RXB01 Cortasetos-Écartement des dents La...   

  id_source      brand_seller productfamily_seller_formatted  \
0    AMZ_FR     SIKAFRANCESAS             colles a carrelage   
1    AMZ_FR  ARISTONTHERMOSPA                                  
2    AMZ_FR               AEG                                  
3    AMZ_FR     SIKAFRANCESAS                         mastic   
4    AMZ_FR     SIKAFRANCESAS                          cires   
5    AMZ_FR     SIKAFRANCESAS                                  
6    AMZ_FR             RYOBI              perceuse visseuse   
7    AMZ_FR     SIKAFRANCESAS          produits d etancheite   
8    AMZ_FR       BLACKDECKER                      arroseurs   
9    AMZ_FR             RYOBI                                  

  prodcategory4_seller_formatted    prodcategory3_seller_formatted  \
0             colles a carrelage                colles et adhesifs   
1                                                                    
2                                 accessoires pour lave vaisselles   
3                         mastic                colles et adhesifs   
4                          cires   peintures teintures et solvants   
5                                                                    
6                      perceuses         outillage electroportatif   
7          produits d etancheite                colles et adhesifs   
8                      arroseurs               materiel d arrosage   
9                                                                    

            prodcategory2_seller_formatted prodcategory1_seller_formatted  \
0                            quincaillerie                      bricolage   
1                                                                       

In [8]:
cols_to_keep = ['productfamily_seller',
                'prodcategory4_seller',
                'prodcategory3_seller',
                'prodcategory2_seller',
                'prodcategory1_seller',
                'productseller_name',
                'id_source',
                'productfamily_seller_stem',
                'prodcategory4_seller_stem',
                'prodcategory3_seller_stem',
                'prodcategory2_seller_stem',
                'prodcategory1_seller_stem',
                'productseller_name_stem']

df2 = df_seller[cols_to_keep]
df2.rename(columns={'productfamily_seller_stem':"prod_fam",
                'prodcategory4_seller_stem':'cat4',
                'prodcategory3_seller_stem':'cat3',
                'prodcategory2_seller_stem':'cat2',
                'prodcategory1_seller_stem':'cat1',
                'productseller_name_stem':'description'}, inplace=True)

In [9]:
df2.head()

productfamily_seller prodcategory4_seller              prodcategory3_seller  \
0   Colles à carrelage   Colles à carrelage                Colles et adhésifs   
1                                                                               
2                                            Accessoires pour lave-vaisselles   
3               Mastic               Mastic                Colles et adhésifs   
4                Cires                Cires  Peintures, teintures et solvants   

                       prodcategory2_seller prodcategory1_seller  \
0                             Quincaillerie            Bricolage   
1                                                                  
2                               Accessoires  Gros électroménager   
3                             Quincaillerie            Bricolage   
4  Peintures, outils et traitement des murs            Bricolage   

                                  productseller_name id_source  \
0  SikaCeram Xtra - Colle à carrelage intérieur e...    AMZ_FR   
1  Ariston 3200362 scaldabagno elettrico PRO 100 H/3    AMZ_FR   
2  AEG Panier À Couverts Lave-vaisselle Cage & Ha...    AMZ_FR   
3              Colle-mastic pour joints d'étanchéité    AMZ_FR   
4  Sikagard Protection Sol MAT - Imperméabilisant...    AMZ_FR   

           prod_fam              cat4                       cat3  \
0  [coll, carrelag]  [coll, carrelag]              [coll, adhes]   
1                []                []                         []   
2                []                []  [accessoir, vaissel, lav]   
3          [mastic]          [mastic]              [coll, adhes]   
4             [cir]             [cir]   [solv, peintur, teintur]   

                           cat2                  cat1  \
0                 [quincailler]            [bricolag]   
1                            []                    []   
2                   [accessoir]  [gros, electromenag]   
3                 [quincailler]            [bricolag]   
4  [trait, peintur, outil, mur]            [bricolag]   

                                         description  
0  [exterieur, max, interieur, xtra, carreau, car...  
1             [ariston, scaldabagno, pro, elettrico]  
2  [vaissel, pani, lav, compart, couvert, cag, ae...  
3                   [coll, etancheit, mastic, joint]  
4  [effet, impermeabilis, mat, sol, sikagard, pro...

In [10]:
#concaténation de tous les champs pertinents en un seul texte par produit
df2['prod_info'] = df2.prod_fam + df2.cat4 +df2.description
df2['prod_info'] = df2['prod_info'].apply(lambda x: ' '.join(x))
df2['prod_info']

0        coll carrelag coll carrelag exterieur max inte...
1                        ariston scaldabagno pro elettrico
2        vaissel pani lav compart couvert cag aeg couve...
3                mastic mastic coll etancheit mastic joint
4        cir cir effet impermeabilis mat sol sikagard p...
5        infiltr fuit terr eau toitur sikagard cuit inc...
6        perceux visseux perceux visseux batter sac rcd...
7        produit etancheit produit etancheit resist dec...
8         arroseur arroseur deck and black arroseur impuls
9              dent ecart longueur ryob rxb cortasetos lam
10       disposit start batter lítio ryob oe demarrage ...
11         pomp fontain pomp cabl pomp norten noir fontain
12       secateur ciseau secateur ciseau ebrancheur dec...
13       accessoir san fil outillag accessoir san fil o...
14       accessoir piec chaudier accessoir piec chaudie...
15                      induct finit plan aeg cuisson noir
16       filtr aspir portat filtr ultraon plus lavabl p.

## TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

#conversion des infos prod en tf-idf et affichage des mots ayant les tf-idf les plus élevés
tvec = TfidfVectorizer(min_df=1, max_df=1.0,ngram_range=(1,1))
tvec_weights = tvec.fit_transform(df2['prod_info'])
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

term    weight
6672         tuil  0.090849
6434         terr  0.066273
1597         cuit  0.065762
5592          riv  0.025655
2949     gouttier  0.024079
32      accessoir  0.021067
3959       mastic  0.020463
2477      faitier  0.016647
5705         roug  0.016528
5232      raccord  0.016275
380        ardois  0.015747
2380    etancheit  0.015319
1349         coll  0.015039
5192          pvc  0.014811
5685         rond  0.014634
1999        droit  0.014208
2199  electroport  0.014179
1893         diam  0.013930
2105          eau  0.013527
1500    couvertur  0.012852

In [40]:
#classement des mots par idf croissant, afin d'identifier des mots peu spécifiques qui pourraient 
#être supprimés lors du préprocessing

idfs = np.asarray(tvec.idf_).ravel().tolist()
idf_df = pd.DataFrame({'term': tvec.get_feature_names(), 'idf': idfs})
idf_df.sort_values(by='idf', ascending=True).head()

term       idf
6672       tuil  1.984445
6434       terr  2.578794
1597       cuit  2.621340
5592        riv  3.377616
32    accessoir  3.382871

Commentaire : l'idf seul ne semble pas permettre d'identifier des mots peu signifiants ; par exemple, "accessoire" a un idf de 0.021, très proche de celui de "mastic", alors que ce dernier mot est beaucoup plus discriminant que "accessoire".

In [18]:
#tfidf version gensim
texts = [[word for word in document.lower().split()] for document in df2['prod_info']]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus)
vectors = tfidf[corpus]

In [21]:
vectors[100]

[(13, 0.1566074490401896),
 (48, 0.34077914189528347),
 (374, 0.7333961761281054),
 (449, 0.2631521988604398),
 (450, 0.42159951946117535),
 (451, 0.2729074891239399)]

## Embeddings

### 1-Latent semantic indexing

#### 1.1 - Détermination de la dimension de l'embedding

In [22]:
n_dim = 1000

In [23]:
lsi = models.LsiModel(vectors, id2word=dictionary, num_topics=n_dim)
corpus_lsi = lsi[vectors]

In [24]:
lsi.show_topics()

[(0,
  '0.555*"cuit" + 0.544*"terr" + 0.404*"tuil" + 0.175*"riv" + 0.148*"roug" + 0.107*"faitier" + 0.104*"douill" + 0.103*"vieil" + 0.103*"droit" + 0.101*"rond"'),
 (1,
  '0.582*"gouttier" + 0.333*"raccord" + 0.276*"pvc" + 0.246*"descent" + 0.200*"coud" + 0.198*"accessoir" + 0.172*"diam" + 0.161*"gris" + 0.147*"girp" + 0.141*"dev"'),
 (2,
  '-0.472*"riv" + -0.335*"tuil" + 0.317*"cuit" + 0.314*"terr" + -0.272*"droit" + -0.265*"rabat" + -0.252*"gauch" + -0.181*"ardois" + -0.143*"roman" + 0.136*"gouttier"'),
 (3,
  '0.476*"mastic" + 0.346*"coll" + 0.342*"etancheit" + 0.228*"sik" + -0.221*"diam" + 0.196*"adhes" + 0.189*"toitur" + -0.163*"douill" + 0.156*"riv" + 0.154*"peintur"'),
 (4,
  '0.402*"diam" + 0.334*"douill" + 0.323*"tuil" + -0.289*"riv" + 0.265*"lantern" + 0.207*"mastic" + -0.203*"gouttier" + -0.185*"droit" + -0.178*"gauch" + -0.175*"rabat"'),
 (5,
  '-0.469*"perceux" + -0.389*"batter" + -0.381*"san" + -0.367*"fil" + -0.275*"visseux" + -0.155*"chargeur" + -0.139*"ryob" + -0.125*

In [25]:
#affichage des valeurs propres pour déterminer le bon nombre de topics
eigenvalues = lsi.projection.s
plt.scatter(range(n_dim),eigenvalues)
plt.xlabel('topic rank')
plt.ylabel('topic eigenvalue')
plt.show()

Commentaire : au vu de ce qui précède, il paraît raisonnable de ne conserver que les 200 premiers topics.

In [26]:
#on refait un embedding lsi en ne gardant que les 200 topics correspondant aux valeurs propres les plus élevées
lsi2 = models.LsiModel(vectors, id2word=dictionary, num_topics=200)
corpus_lsi2 = lsi2[vectors]

#### 1.2 - Importation du train set et représentation dans l'embedding

In [38]:
chemin = "C:/Users/JN/Desktop/Arbeit/fil_rouge/brouillons/"
data = pd.read_csv("C:/Users/JN/Desktop/Arbeit/fil_rouge/brouillons/20190115_train_set.csv",
                   index_col=0)

data.columns

Index(['productfamily_seller', 'productseller_name', 'id_source',
       'max_sim_prod_fam', 'max_occurences_prod_fam', 'best_match_prod_fam',
       'ex_aequos_prod_fam', 'prod_fam', 'cat4', 'cat3', 'cat2', 'cat1',
       'description', 'max_sim_description', 'max_occurences_description',
       'best_match_description', 'ex_aequos_description', 'prod_fam.1',
       'description.1', 'max_sim_cat4', 'max_occurences_cat4',
       'best_match_cat4', 'ex_aequos_cat4', 'prod_fam.2', 'description.2',
       'best_match', 'similarity'],
      dtype='object')

In [39]:
data.head()

productfamily_seller                                 productseller_name  \
6     Perceuse-visseuse  Perceuse-visseuse RYOBI 18V OnePlus - 1 batter...   
25  Marteau perforateur  AEG Powertools Marteau Perforateur BBH 18 (PRO...   
38         Taille-haies   Ryobi 5133002113 RHT36C5525 Taille-haie sans fil   
59              Bobines  lot therm filme 3 bobines + fil 1,6mm pour gam...   
60       Pulvérisateurs  Ryobi - OWS1880 One - Pulvérisateur pour déshe...   

   id_source  max_sim_prod_fam  max_occurences_prod_fam  \
6     AMZ_FR               0.5                        1   
25    AMZ_FR               0.5                        1   
38    AMZ_FR               1.0                        1   
59    AMZ_FR               0.0                      105   
60    AMZ_FR               1.0                        1   

                          best_match_prod_fam  \
6   Perceuse, perforateur et visseuse filaire   
25                                    Marteau   
38                               Taille-haies   
59                                   no match   
60                              Pulvérisation   

                               ex_aequos_prod_fam                prod_fam  \
6   ['Perceuse, perforateur et visseuse filaire']  {'perceux', 'visseux'}   
25                                    ['Marteau']   {'perfor', 'marteau'}   
38                               ['Taille-haies']        {'taill', 'hai'}   
59                                   ['no match']               {'bobin'}   
60                              ['Pulvérisation']            {'pulveris'}   

                               cat4  \
6                       {'perceux'}   
25                      {'perceux'}   
38                 {'taill', 'hai'}   
59  {'bordur', 'accessoir', 'coup'}   
60                     {'pulveris'}   

                                                 cat3  \
6                         {'outillag', 'electroport'}   
25                        {'outillag', 'electroport'}   
38                  {'outillag', 'motoris', 'jardin'}   
59                {'outillag', 'accessoir', 'jardin'}   
60  {'protect', 'contr', 'le', 'lutt', 'nuisibl', ...   

                                            cat2          cat1  \
6            {'main', 'outillag', 'electroport'}  {'bricolag'}   
25           {'main', 'outillag', 'electroport'}  {'bricolag'}   
38  {'outillag', 'motoris', 'tondeux', 'jardin'}    {'jardin'}   
59  {'outillag', 'motoris', 'tondeux', 'jardin'}    {'jardin'}   
60                                  {'jardinag'}    {'jardin'}   

                                          description  max_sim_description  \
6   {'batter', 'oneplus', 'perceux', 'rcd', 'visse...             0.222222   
25  {'bbh', 'powertool', 'perfor', 'marteau', 'ion...             0.142857   
38     {'san', 'ryob', 'taill', 'fil', 'rhtc', 'hai'}             0.333333   
59  {'ryob', 'rac', 'fil', 'therm', 'film', 'lot',...             0.076923   
60  {'vert', 'pulveris', 'ryob', 'desherbag', 'one...             0.166667   

    max_occurences_description                      best_match_description  \
6                            1                        Batterie et chargeur   
25                           1                                     Marteau   
38                           1                                Taille-haies   
59                           1  Perceuse, perforateur et visseuse sans fil   
60                           1                               Pulvérisation   

                             ex_aequos_description              prod_fam.1  \
6                         ['Batterie et chargeur']  {'perceux', 'visseux'}   
25                                     ['Marteau']   {'perfor', 'marteau'}   
38                                ['Taille-haies']        {'taill', 'hai'}   
59  ['Perceuse, perforateur et visseuse sans fil']               {'bobin'}   
60                               ['Pulvérisation']            {'pulveris'}   

                                       

In [65]:
data2 = data[['productfamily_seller', 
             'productseller_name', 
             'id_source','prod_fam', 
             'cat4','description',
             'best_match', 
             'similarity']]

#cleaning
data2.prod_fam.replace({'{':'','}':'',"'":'',',':''},regex=True,inplace=True)
data2.cat4.replace({'{':'','}':'',"'":'',',':''},regex=True,inplace=True)
data2.description.replace({'{':'','}':'',"'":'',',':''},regex=True,inplace=True)

#concatenation des differents champs de description en un seul champ (cf plus haut)
data2['prod_info'] = data2.prod_fam + ' ' + data2.cat4 + ' '+ data2.description
# data2['prod_info'] = data2['prod_info'].apply(lambda x: ' '.join(x))
data2.head()

productfamily_seller                                 productseller_name  \
6     Perceuse-visseuse  Perceuse-visseuse RYOBI 18V OnePlus - 1 batter...   
25  Marteau perforateur  AEG Powertools Marteau Perforateur BBH 18 (PRO...   
38         Taille-haies   Ryobi 5133002113 RHT36C5525 Taille-haie sans fil   
59              Bobines  lot therm filme 3 bobines + fil 1,6mm pour gam...   
60       Pulvérisateurs  Ryobi - OWS1880 One - Pulvérisateur pour déshe...   

   id_source         prod_fam                   cat4  \
6     AMZ_FR  perceux visseux                perceux   
25    AMZ_FR   perfor marteau                perceux   
38    AMZ_FR        taill hai              taill hai   
59    AMZ_FR            bobin  bordur accessoir coup   
60    AMZ_FR         pulveris               pulveris   

                                          description  \
6   batter oneplus perceux rcd visseux chargeur ry...   
25           bbh powertool perfor marteau ion pro aeg   
38                        san ryob taill fil rhtc hai   
59         ryob rac fil therm film lot gamm bobin one   
60                vert pulveris ryob desherbag one ow   

                                   best_match  similarity  \
6   Perceuse, perforateur et visseuse filaire         0.5   
25                                    Marteau         0.5   
38                               Taille-haies         1.0   
59         Débroussailleuse et coupe-bordures         0.5   
60                              Pulvérisation         1.0   

                                            prod_info  
6   perceux visseux perceux batter oneplus perceux...  
25  perfor marteau perceux bbh powertool perfor ma...  
38    taill hai taill hai san ryob taill fil rhtc hai  
59  bobin bordur accessoir coup ryob rac fil therm...  
60  pulveris pulveris vert pulveris ryob desherbag...

In [88]:
#affichons à nouveau la liste des label par ordre décroissant d'occurrences
cat_count = data2.groupby('best_match').agg('count')
cat_count = cat_count['prod_info'].sort_values( ascending=False)
cat_count.head(30)

best_match
Gouttière, descente et accessoires                        985
Mastic colle Etanchéité                                   663
Couverture                                                615
Accessoires et consommables outillage électroportatif     599
Tondeuses electriques, thermiques et autoportée           562
Accessoires chauffe-eau                                   387
Perceuse, perforateur et visseuse sans fil                322
Batterie et chargeur                                      322
Perceuse, perforateur et visseuse filaire                 200
Conduit et accessoires d'aération                         196
Ponceuse électrique                                       173
Meuleuse et rainureuse                                    161
Aérateur et VMC                                           157
Taille-haies                                              138
Tronçonneuse, scie et fendeur de bois                     128
chauffe-eau électrique                                    1

In [146]:
#Comme les différentes catégories ne sont pas représentées de manière équilibrée, 
#on ne va dans un premier temps travailler que sur celles qui contiennent au moins 100 points de données.
eff_min = 0
selected_cats = list(cat_count[cat_count>eff_min].index.values)
data3 = data2[data2['best_match'].isin(selected_cats)]

In [147]:
# on transforme en tfidf
texts = [[word for word in document.lower().split()] for document in data3['prod_info']]
corpus = [dictionary.doc2bow(text) for text in texts]
data_tfidf = tfidf[corpus]

In [148]:
#on exprime les tfidf dans l'embedding issu de la lsi
data_lsi2 = lsi2[data_tfidf]
data_lsi2[0]

[(0, 0.0010993588230800765),
 (1, 0.013298314791875642),
 (2, -0.01645087496651397),
 (3, 0.05869177471641013),
 (4, 0.08613553532914192),
 (5, -0.5586376463996932),
 (6, 0.06762889584708368),
 (7, 0.09242226929334835),
 (8, 0.003020033034908033),
 (9, 0.03604855632212265),
 (10, 0.002841689919864988),
 (11, -0.014857293242918294),
 (12, 0.017826060476188607),
 (13, -0.14697100917532813),
 (14, 0.0024959023695598406),
 (15, -0.0757332306356843),
 (16, 0.009900840665510127),
 (17, 0.020820723667788626),
 (18, 0.06341272245103516),
 (19, 0.014891539360948457),
 (20, 0.021913177359508387),
 (21, 0.003662939585342889),
 (22, -0.00961113088924424),
 (23, 0.003195139229261286),
 (24, 0.00908456692935235),
 (25, -0.014839147689533184),
 (26, 0.0005172577088319919),
 (27, -0.020146156009798962),
 (28, -0.005131070949575809),
 (29, 0.010614427977012914),
 (30, 0.0021669665983404347),
 (31, -0.0027269373543382103),
 (32, 0.0002165368073900574),
 (33, -0.03414065761472364),
 (34, -0.0170686641333

In [149]:
#On stocke enfin ces nouveaux objets dans un dataframe
from gensim import matutils
X = pd.DataFrame(matutils.corpus2csc(data_lsi2).T.todense(),index=data3.index.values)

#et on crée un dictionnaire des catégories:
label_dict = dict(zip(selected_cats, range(len(selected_cats))))

#et, à toutes fins utiles, un dictionnaire des topics:
topic_dict = dict(lsi2.show_topics())

In [150]:
#on convertit les labels dans le nouveau format numérique
Y = data3['best_match'].apply(lambda x: label_dict[x])

In [151]:
#train test split
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,test_size=0.2)

In [152]:
#entraînement d'un modèle
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb = XGBClassifier(n_jobs=-1)
xgb.fit(Xtrain, Ytrain)
Ypred = xgb.predict(Xtest)
score = accuracy_score(Ytest, Ypred)
score

0.9871611982881597

Voyons à présent les résultats que donne ce classifieur sur un petit échantillon de données qui n'ont pas été labellisées.

In [162]:
#Nous allons tout d'abord extraire les données non pré-labellisées dans un dataframe à part:
indices_labelled = X.index.values
indices_unlabelled = [idx for idx in df_seller.index.values if idx not in indices_labelled]
df_unlabelled = df2.loc[indices_unlabelled]
df_unlabelled.head()

#on y applique ensuite les mêmes transformations qu'aux autres df (tf-idf+lsi):
texts = [[word for word in document] for document in df_unlabelled['prod_info']]
corpus = [dictionary.doc2bow(text) for text in texts]
data_unlabelled_tfidf = tfidf[corpus]
data_unlabelled_lsi2 = lsi2[data_unlabelled_tfidf]

#on stocke les vecteurs dans un dataframe
X_unlabelled = pd.DataFrame(matutils.corpus2csc(data_unlabelled_lsi2).T.todense(),index=df_unlabelled.index.values)

X_unlabelled.head()

0         1         2         3         4         5         6    \
0  0.001803  0.059278  0.026676  0.159887  0.061036  0.024375  0.026227   
1  0.000160  0.004785  0.001167  0.015617  0.010374 -0.005150 -0.002552   
2  0.001538  0.001249 -0.000156  0.001283  0.005083 -0.017104  0.002228   
3  0.006857  0.119216  0.079227  0.625998  0.257460  0.115400  0.053632   
4  0.001596  0.009791  0.008269  0.052962  0.020913  0.007768 -0.001321   

        7         8         9         10        11        12        13   \
0  0.010803  0.069340 -0.086917 -0.016528  0.022340  0.083761  0.002866   
1 -0.022721  0.007735 -0.009095  0.002356  0.005409  0.004599  0.002058   
2 -0.000356  0.000980 -0.000860  0.002425  0.004560 -0.002452  0.002952   
3  0.002952  0.133725 -0.200949 -0.086001 -0.000004  0.021277 -0.020281   
4 -0.007692 -0.018474  0.058380  0.017694 -0.000741 -0.022208 -0.001084   

        14        15        16        17        18        19        20   \
0 -0.005306  0.005343 -0.006446 -0.010502 -0.008400  0.000501 -0.001872   
1  0.001077  0.002837 -0.000396  0.016258  0.006359  0.008611  0.009403   
2  0.000096  0.002538 -0.004755 -0.004288  0.006242  0.002300 -0.008850   
3 -0.003428 -0.010533  0.007515 -0.010850  0.000500 -0.008822  0.028879   
4  0.004308 -0.002303  0.003254 -0.000184  0.001925 -0.002952  0.005406   

        21        22        23        24        25        26        27   \
0  0.030749  0.011781 -0.007813 -0.169570 -0.009940  0.003121  0.000283   
1  0.015640 -0.000317  0.001596 -0.011454  0.001370 -0.003873 -0.008534   
2  0.002688  0.001536  0.003733  0.001874 -0.011983 -0.002375  0.000235   
3 -0.021539  0.003984  0.005541 -0.075425 -0.017843  0.061295  0.133036   
4  0.001187 -0.003712  0.004759  0.051658  0.011871 -0.026201 -0.065775   

        28        29     ...          170       171       172       173  \
0 -0.001033  0.039008    ...     0.016675 -0.043063 -0.019333 -0.042066   
1 -0.001022 -0.001398    ...     0.022944  0.063739  0.024061 -0.009641   
2  0.007395  0.000100    ...    -0.003942 -0.007110 -0.013563 -0.006892   
3 -0.006723 -0.036816    ...    -0.036672 -0.005353 -0.016397 -0.003861   
4 -0.007324  0.033122    ...    -0.025405  0.024274  0.025468 -0.007713   

        174       175       176       177       178       179       180  \
0 -0.014353 -0.056737 -0.023724 -0.021790 -0.016064  0.010577 -0.019745   
1 -0.021412  0.009970  0.015163 -0.025818  0.000501 -0.010412 -0.002969   
2  0.013573  0.004421  0.000143  0.009746  0.005863 -0.007805 -0.001008   
3  0.014931  0.001178 -0.016159 -0.008884 -0.017942 -0.016278  0.001536   
4  0.028741  0.015693  0.023577 -0.028111 -0.019756 -0.024686 -0.012373   

        181       182       183       184       185       186       187  \
0  0.002977  0.006670 -0.015533 -0.017922  0.002396 -0.018496 -0.009963   
1  0.012350  0.011049  0.049563  0.046205  0.027652 -0.021123  0.050959   
2 -0.003580 -0.001682  0.005271  0.001141  0.002292 -0.005890  0.002336   
3  0.012932  0.016673 -0.010983 -0.015696  0.019524  0.006699 -0.003634   
4 -0.010794  0.031528  0.023913  0.025159  0.017585  0.007078  0.009077   

        188       189       190       191       192       193       194  \
0  0.018809  0.010534 -0.034802 -0.050751 -0.006346  0.002512  0.010836   
1  0.012466  0.055180  0.029507  0.038493 -0.016669 -0.037635 -0.011584   
2 -0.002052  0.016637  0.011786  0.001613 -0.004488  0.006052  0.003208   
3  0.017869 -0.018483 -0.018963 -0.013898  0.006349 -0.011251  0.000585   
4  0.020320  0.014922  0.007949  0.014957 -0.006878 -0.000470 -0.016253   

        195       196       197       198       199  
0  0.023766 -0.007097 -0.009601  0.018806 -0.040324  
1  0.037002  0.003795  0.046154 -0.018464  0.001147  
2  0.005276 -0.001817  0.001104 -0.006693  0.008864  
3  0.008793 -0.014755 -0.026127  0.000100  0.004462  
4  0.023236 -0.029569  0.004898  0.004629 -0.030624  

[5 rows x 200 columns]

In [164]:
#on extrait ensuite 1000 lignes au hasard, auxquelles on applique le classifieur
sample_idx = np.random.choice(X_unlabelled.index.values, (1000,), replace=False)
X_valid = X_unlabelled.loc[sample_idx]
Y_pred_valid = xgb.predict(X_valid)

In [165]:
#on affiche enfin les prédictions avec les données d'origine dans un dataframe, 
#pour évaluer les résultats du modèle

(1000,)

In [161]:
df_unlabelled.head()

productfamily_seller prodcategory4_seller              prodcategory3_seller  \
0   Colles à carrelage   Colles à carrelage                Colles et adhésifs   
1                                                                               
2                                            Accessoires pour lave-vaisselles   
3               Mastic               Mastic                Colles et adhésifs   
4                Cires                Cires  Peintures, teintures et solvants   

                       prodcategory2_seller prodcategory1_seller  \
0                             Quincaillerie            Bricolage   
1                                                                  
2                               Accessoires  Gros électroménager   
3                             Quincaillerie            Bricolage   
4  Peintures, outils et traitement des murs            Bricolage   

                                  productseller_name id_source  \
0  SikaCeram Xtra - Colle à carrelage intérieur e...    AMZ_FR   
1  Ariston 3200362 scaldabagno elettrico PRO 100 H/3    AMZ_FR   
2  AEG Panier À Couverts Lave-vaisselle Cage & Ha...    AMZ_FR   
3              Colle-mastic pour joints d'étanchéité    AMZ_FR   
4  Sikagard Protection Sol MAT - Imperméabilisant...    AMZ_FR   

           prod_fam              cat4                       cat3  \
0  [coll, carrelag]  [coll, carrelag]              [coll, adhes]   
1                []                []                         []   
2                []                []  [accessoir, vaissel, lav]   
3          [mastic]          [mastic]              [coll, adhes]   
4             [cir]             [cir]   [solv, peintur, teintur]   

                           cat2                  cat1  \
0                 [quincailler]            [bricolag]   
1                            []                    []   
2                   [accessoir]  [gros, electromenag]   
3                 [quincailler]            [bricolag]   
4  [trait, peintur, outil, mur]            [bricolag]   

                                         description  \
0  [exterieur, max, interieur, xtra, carreau, car...   
1             [ariston, scaldabagno, pro, elettrico]   
2  [vaissel, pani, lav, compart, couvert, cag, ae...   
3                   [coll, etancheit, mastic, joint]   
4  [effet, impermeabilis, mat, sol, sikagard, pro...   

                                           prod_info  
0  [coll, carrelag, coll, carrelag, exterieur, ma...  
1             [ariston, scaldabagno, pro, elettrico]  
2  [vaissel, pani, lav, compart, couvert, cag, ae...  
3   [mastic, mastic, coll, etancheit, mastic, joint]  
4  [cir, cir, effet, impermeabilis, mat, sol, sik...